In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 0
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC10(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC10(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC10(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


D:\Hu_code\ResNet18_ESC10\ESC10_kFold_1s\..\TrainDataPreprocessing.py:64: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 25.72 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []


#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    #------------------------------------------------
    #需要調整的地方
    #------------------------------------------------
    #學習率
    lr= 0.0032
    #epoch
    epoch= 90
    #訓練資料(要用幾秒訓練) data_1S/data_3S/data_5S
    trainData = data_1S
    #------------------------------------------------
    
    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型
    model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
    #測試準確度
    test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
    test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
    test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    training_time = end_time - start_time

    #印出訓練耗時
    print("Training Time: {:.2f} seconds".format(training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("訓練過程\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
    f.write("Training Time: {:.2f} seconds\n".format(training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")

    #--------------------------------------------------------------------

    
    kFoldTrainTime.append(training_time)
    kFoldTrainAccuracy.append(train_accuracy[-1])
    kFoldTestAccuracy1.append(test_accuracy1)
    kFoldTestAccuracy3.append(test_accuracy3)
    kFoldTestAccuracy5.append(test_accuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1
Train Epoch: 1/90 Train_Loss: 1.5444140434265137 Train_acc: 30.187500%
Train Epoch: 2/90 Train_Loss: 1.3571827411651611 Train_acc: 51.750000%
Train Epoch: 3/90 Train_Loss: 1.3521277904510498 Train_acc: 58.875000%
Train Epoch: 4/90 Train_Loss: 0.8422244787216187 Train_acc: 67.250000%
Train Epoch: 5/90 Train_Loss: 0.5761439800262451 Train_acc: 72.250000%
Train Epoch: 6/90 Train_Loss: 0.617663562297821 Train_acc: 74.875000%
Train Epoch: 7/90 Train_Loss: 0.5110177993774414 Train_acc: 76.687500%
Train Epoch: 8/90 Train_Loss: 0.7181369066238403 Train_acc: 77.875000%
Train Epoch: 9/90 Train_Loss: 0.5528047680854797 Train_acc: 79.687500%
Train Epoch: 10/90 Train_Loss: 0.4305826425552368 Train_acc: 82.062500%
Train Epoch: 11/90 Train_Loss: 0.49568098

Train Epoch: 16/90 Train_Loss: 0.3449755609035492 Train_acc: 90.375000%
Train Epoch: 17/90 Train_Loss: 0.5118244290351868 Train_acc: 91.937500%
Train Epoch: 18/90 Train_Loss: 0.2702639400959015 Train_acc: 91.375000%
Train Epoch: 19/90 Train_Loss: 0.09308566898107529 Train_acc: 92.562500%
Train Epoch: 20/90 Train_Loss: 0.1667831391096115 Train_acc: 92.562500%
Train Epoch: 21/90 Train_Loss: 0.23146477341651917 Train_acc: 92.187500%
Train Epoch: 22/90 Train_Loss: 0.20913554728031158 Train_acc: 92.000000%
Train Epoch: 23/90 Train_Loss: 0.12044006586074829 Train_acc: 91.437500%
Train Epoch: 24/90 Train_Loss: 0.16282092034816742 Train_acc: 90.562500%
Train Epoch: 25/90 Train_Loss: 0.20039434731006622 Train_acc: 91.500000%
Train Epoch: 26/90 Train_Loss: 0.1020713821053505 Train_acc: 92.562500%
Train Epoch: 27/90 Train_Loss: 0.09456560015678406 Train_acc: 92.125000%
Train Epoch: 28/90 Train_Loss: 0.15911170840263367 Train_acc: 92.750000%
Train Epoch: 29/90 Train_Loss: 0.12474930286407471 Train

Train Epoch: 34/90 Train_Loss: 0.12821361422538757 Train_acc: 94.625000%
Train Epoch: 35/90 Train_Loss: 0.11288534849882126 Train_acc: 94.625000%
Train Epoch: 36/90 Train_Loss: 0.11016615480184555 Train_acc: 94.750000%
Train Epoch: 37/90 Train_Loss: 0.15309926867485046 Train_acc: 94.375000%
Train Epoch: 38/90 Train_Loss: 0.24767489731311798 Train_acc: 94.812500%
Train Epoch: 39/90 Train_Loss: 0.14758938550949097 Train_acc: 94.750000%
Train Epoch: 40/90 Train_Loss: 0.1780741810798645 Train_acc: 94.375000%
Train Epoch: 41/90 Train_Loss: 0.14311803877353668 Train_acc: 94.750000%
Train Epoch: 42/90 Train_Loss: 0.1767914742231369 Train_acc: 94.687500%
Train Epoch: 43/90 Train_Loss: 0.07155342400074005 Train_acc: 94.812500%
Train Epoch: 44/90 Train_Loss: 0.1451255828142166 Train_acc: 94.750000%
Train Epoch: 45/90 Train_Loss: 0.13130953907966614 Train_acc: 94.437500%
Train Epoch: 46/90 Train_Loss: 0.12572231888771057 Train_acc: 94.750000%
Train Epoch: 47/90 Train_Loss: 0.11888295412063599 Tra

Train Epoch: 52/90 Train_Loss: 0.1806797981262207 Train_acc: 94.437500%
Train Epoch: 53/90 Train_Loss: 0.19496884942054749 Train_acc: 94.562500%
Train Epoch: 54/90 Train_Loss: 0.11163801699876785 Train_acc: 94.500000%
Train Epoch: 55/90 Train_Loss: 0.08718149363994598 Train_acc: 94.125000%
Train Epoch: 56/90 Train_Loss: 0.14152304828166962 Train_acc: 94.625000%
Train Epoch: 57/90 Train_Loss: 0.15202747285366058 Train_acc: 94.562500%
Train Epoch: 58/90 Train_Loss: 0.06894804537296295 Train_acc: 94.875000%
Train Epoch: 59/90 Train_Loss: 0.13052937388420105 Train_acc: 94.312500%
Train Epoch: 60/90 Train_Loss: 0.2916842997074127 Train_acc: 94.187500%
Train Epoch: 61/90 Train_Loss: 0.07157257944345474 Train_acc: 94.687500%
Train Epoch: 62/90 Train_Loss: 0.030266452580690384 Train_acc: 94.375000%
Train Epoch: 63/90 Train_Loss: 0.08097323030233383 Train_acc: 95.000000%
Train Epoch: 64/90 Train_Loss: 0.061162494122982025 Train_acc: 94.562500%
Train Epoch: 65/90 Train_Loss: 0.08712445944547653 

Train Epoch: 70/90 Train_Loss: 0.11973631381988525 Train_acc: 93.437500%
Train Epoch: 71/90 Train_Loss: 0.18124571442604065 Train_acc: 93.812500%
Train Epoch: 72/90 Train_Loss: 0.18693052232265472 Train_acc: 94.000000%
Train Epoch: 73/90 Train_Loss: 0.19401253759860992 Train_acc: 94.250000%
Train Epoch: 74/90 Train_Loss: 0.3006804883480072 Train_acc: 94.250000%
Train Epoch: 75/90 Train_Loss: 0.1715267449617386 Train_acc: 94.250000%
Train Epoch: 76/90 Train_Loss: 0.0884142741560936 Train_acc: 93.500000%
Train Epoch: 77/90 Train_Loss: 0.10629858821630478 Train_acc: 94.375000%
Train Epoch: 78/90 Train_Loss: 0.0721309557557106 Train_acc: 94.375000%
Train Epoch: 79/90 Train_Loss: 0.07363872975111008 Train_acc: 94.187500%
Train Epoch: 80/90 Train_Loss: 0.12898781895637512 Train_acc: 94.312500%
Train Epoch: 81/90 Train_Loss: 0.20113874971866608 Train_acc: 94.250000%
Train Epoch: 82/90 Train_Loss: 0.12556077539920807 Train_acc: 94.125000%
Train Epoch: 83/90 Train_Loss: 0.06621474772691727 Trai

In [5]:
#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracy[fold], kFoldTestAccuracy1[fold], kFoldTestAccuracy3[fold], kFoldTestAccuracy5[fold], kFoldTrainTime[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracy), max(kFoldTestAccuracy1), max(kFoldTestAccuracy3), max(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracy), min(kFoldTestAccuracy1), min(kFoldTestAccuracy3), min(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracy), np.mean(kFoldTestAccuracy1), np.mean(kFoldTestAccuracy3), np.mean(kFoldTestAccuracy5)))

f.close()